In [1]:
j = {'status': 'PENDING', 'job_id': 'verification-utc_comparison_sbml-7ae8dceb-2f4b-4f22-855a-48e77af67453', 'comparison_id': 'utc_comparison_sbml', 'path': 'file_uploads/verification-utc_comparison_sbml-7ae8dceb-2f4b-4f22-855a-48e77af67453/BIOMD0000000001_url.xml', 'simulators': ['copasi', 'pysces', 'tellurium'], 'timestamp': '2024-09-03 22:30:42.425126', 'start': 0, 'end': 10, 'steps': 100, 'include_outputs': True, 'expected_results': None, 'rTol': None, 'aTol': None, 'selection_list': None}
start = 0 
dur = 1
steps = 10

In [2]:
from workers import VerificationWorker

w = VerificationWorker(j)
sbml_fp = '/Users/alexanderpatrie/Desktop/repos/bio-check/model-examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'smoldyn-process',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator',
  'smoldyn-step',
  'simularium-smoldyn-step',
  'database-emitter']


In [ ]:
await w.run()
    

In [4]:
w.job_result.get('rmse')

{'error': '(\'rmse_calculation error:\\n\'\n \'Traceback (most recent call last):\\n\'\n \'  File \'\n \'"/Users/alexanderpatrie/Desktop/repos/bio-check/compose_worker/workers.py", \'\n \'line 130, in run\\n\'\n "    self.job_result[\'rmse\'] = self._calculate_pairwise_rmse()\\n"\n \'  File \'\n \'"/Users/alexanderpatrie/Desktop/repos/bio-check/compose_worker/workers.py", \'\n \'line 231, in _calculate_pairwise_rmse\\n\'\n \'    mse_vals = mse_values[i][j]\\n\'\n "TypeError: \'float\' object is not subscriptable\\n")'}

In [ ]:
import pprint

x = pprint.pformat('hello')

In [17]:
spec_data = w.job_result
simulators = w.job_params['simulators']
import numpy as np
import pandas as pd

self = w 
spec_data = self.job_result
simulators = self.job_params['simulators']
if self.job_params.get('expected_results') is not None:
    simulators.append('expected_results')
n = len(simulators)
# set up empty matrix
rmse_matrix = np.zeros((n, n))

In [48]:
# enumerate over i,j of simulators in a matrix
import math
cols = []
mse_values = []
for i, sim_i in enumerate(simulators):
    for j, sim_j in enumerate(simulators):
        if i != j:
            # fetch mse values 
            for observable, observable_data in spec_data.items():
                if not isinstance(observable_data, str):
                    mse_data = observable_data.get('mse', {})
                    # means that simulator was successful and is in mse matr
                    if sim_j in mse_data.keys():
                        # append sim_j to cols because it is valid (in the m
                        cols.append(sim_j)
                        
                    # mse_data[sim_j] is a dict containing MSEs with oth
                    sim_i_output = list(mse_data.get(sim_i, {}).values())
                    sim_j_output = list(mse_data.get(sim_j, {}).values())
                    
                    for comparison_sim, mse_value in mse_data.get(sim_j, {}).items():
                        if comparison_sim == sim_i:
                            mse_values.append(sim_i_output)
                            # mse_values.append(mse_value)
                                
            # # get unique list of valid sim keys and adjust matrix shape ac
            # cols = list(set(cols))
            # n_valid_simulators = len(cols)
            # adjusted_matrix_shape = (n_valid_simulators, n_valid_simulator
            # rmse_matrix = np.resize(rmse_matrix, adjusted_matrix_shape)
            # # assign mse_values to newly shaped matrix (should be same sha
            # if mse_values:
            #     mean_mse = sum(mse_values) / len(mse_values)
            #     rmse_matrix[i, j] = math.sqrt(mean_mse)
            # else:
                # TODO: make this more robust
                # rmse_matrix[i, j] = np.nan
        # else:
            # rmse_matrix[i, j] = 0.0
# get unique list of valid sim keys and adjust matrix shape accordingly 
columns = list(set(cols))
n_valid_simulators = len(columns)
adjusted_matrix_shape = (n_valid_simulators, n_valid_simulators)
rmse_matrix = np.resize(rmse_matrix, adjusted_matrix_shape)
print(f'cols: {columns}')
print(f'shape: {rmse_matrix.shape}')

# assign mse_values to newly shaped matrix (should be same shape)
if mse_values:
    for i, sim_i in enumerate(columns):
        print('i', i)
        for j, sim_output in enumerate(mse_values):
            print('j', j)
            
            print('mse', mse_values[i])
            if not j == i:
                print('...')
                print(len(mse_values))
                mse_vals = mse_values[i][j]
                mean_mse = sum(mse_vals) / len(mse_vals)
                rmse_matrix[i, j] = math.sqrt(mean_mse)

return pd.DataFrame(rmse_matrix, columns=columns, index=columns).to_dict()

cols: ['copasi', 'tellurium']
shape: (2, 2)
i 0
j 0
mse [0.0, 3.6024572241473137e-13]
j 1
mse [0.0, 3.6024572241473137e-13]
...
24


TypeError: 'float' object is not iterable

In [ ]:
spec_data

In [ ]:
rmse_matrix.squeeze()

In [ ]:
rmse_matrix.resize((2, 2))

In [ ]:
rmse_matrix = np.resize(rmse_matrix, (2, 2))

In [ ]:
rmse_matrix